In [1]:
#my modules
import wrangle as wr
import explore as ex
import model as m

#Imports
import pandas as pd
import numpy as np

import nltk
# nltk.download('words')
# words = set(nltk.corpus.words.words())

imports loaded successfully, awaiting commands...


In [2]:
pd.set_option("display.max_colwidth", 250)

In [3]:
df = pd.read_parquet('cfpb_prep.parquet')

In [4]:
df.company_response_to_consumer.value_counts()

Closed with explanation            982289
Closed with non-monetary relief    205284
Closed with monetary relief         50963
Untimely response                    4459
Closed                               3741
Name: company_response_to_consumer, dtype: int64

In [5]:
relief = ['Closed with monetary relief', 'Closed with non-monetary relief']
no_relief = ['Closed with explanation']
df = df[df['company_response_to_consumer'] != "Untimely response"]
df = df[df['company_response_to_consumer'] != "Closed"]

In [6]:
df['response'] = np.where(df['company_response_to_consumer'].isin(relief),'relief','')
df['response'] = np.where(df['company_response_to_consumer'].isin(no_relief),'no_relief',df['response'])

In [7]:
df = df.drop(columns='company_response_to_consumer')

In [8]:
df.response.value_counts(normalize=True)

no_relief    0.793105
relief       0.206895
Name: response, dtype: float64

In [9]:
df.response.value_counts()

no_relief    982289
relief       256247
Name: response, dtype: int64

Specificity: When the actual value is negative, how often is the prediction correct?

* Something we want to maximize
* How "specific" (or "selective") is the classifier in predicting positive instances?
* TN / all negative
    * all negative = TN + FP
    
False Positive Rate: When the actual value is negative, how often is the prediction incorrect?

In [10]:
# split
train, val, test = wr.split_data(df, 'response')

Prepared DF: (1238536, 8)
Train: (743121, 8)
Validate: (247707, 8)
Test: (247708, 8)


In [25]:
train.isnull().value_counts()

date_received  company_name  state  tags   product_bins  clean  lemon  response
False          False         False  False  False         False  False  False       743093
                                                         True   True   False           24
                                                         False  True   False            4
dtype: int64

In [11]:
train.head(1)

,date_received,company_name,state,tags,product_bins,clean,lemon,response
274292,2019-09-09,Experian Information Solutions Inc.,FL,Servicemember,credit_report,i have been a victim of id theft and have submitted documentation to the credit bureaus and they are refusing to abide by fcra b pertaining to my request the original creditor has no proof other than statements and i have constantly refuted this ...,victim id theft submitted documentation credit bureau refusing abide fcra b pertaining request original creditor proof statement constantly refuted charge submitting proof id theft claim reporting complaint credit bureau consistent harm financial...,relief


In [13]:
def encode(df):
    '''Encode categorical columns'''
    # columns to encode
    cols = ['tags','product_bins']
    # encode the dummies
    dummy = pd.get_dummies(df[cols],prefix='',prefix_sep='',drop_first=True)
    # bring the dummies along
    return pd.concat([df,dummy],axis=1)


def process_data_modeling(train, validate, test):
    """
    The function `process_data_modeling` reads data from parquet files, performs data sampling,
    encoding, and splits the data into training, validation, and test sets.
    
    :param train: The `train` parameter is the training dataset, which is a pandas DataFrame containing
    the data for training the model
    :param validate: The `validate` parameter is a DataFrame that contains the validation data. It is
    read from a parquet file named 'validate.parquet'
    :param test: The `test` parameter is a DataFrame that contains the test data for your model. It is
    read from a parquet file named 'test.parquet'
    :return: six variables: X_train, y_train, X_val, y_val, X_test, and y_test.
    """
    random_state = 123
    response_categories = [
        'relief',
        'no_relief'
    ]

    sm_train = []
    sm_val = []
    sm_test = []

    small_train = pd.DataFrame()
    small_val = pd.DataFrame()
    small_test = pd.DataFrame()

    random_state = 123
    percent = .2

    for category in response_categories:
        sm_train.append(int(round(len(train[train.response == category]) * percent, 0)))
        sm_val.append(int(round(len(validate[validate.response == category]) * percent, 0)))
        sm_test.append(int(round(len(test[test.response == category]) * percent, 0)))
        
        small_train = small_train.append(train[train.response == category].sample(sm_train[-1], random_state=random_state))
        small_val = small_val.append(validate[validate.response == category].sample(sm_val[-1], random_state=random_state))
        small_test = small_test.append(test[test.response == category].sample(sm_test[-1], random_state=random_state))

    small_train.reset_index(drop=True, inplace=True)
    small_val.reset_index(drop=True, inplace=True)
    small_test.reset_index(drop=True, inplace=True)

    X_train = encode(small_train)
    X_train = X_train.drop(columns=['date_received','clean','state','company_name','tags','product_bins', 'response'])
    y_train = small_train['response']
    X_val = encode(small_val)
    X_val = X_val.drop(columns=['date_received','clean','state','company_name','tags','product_bins', 'response'])
    y_val = small_val['response']
    X_test = encode(small_test)
    X_test = X_test.drop(columns=['date_received', 'clean','state','company_name','tags','product_bins', 'response'])
    y_test = small_test['response']

    return X_train, y_train, X_val, y_val, X_test, y_test

In [14]:
# X_train, etc...
X_train, y_train, X_val, y_val, X_test, y_test = process_data_modeling(train, val, test)

In [15]:
# vectorizing
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

# decision tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report, recall_score, average_precision_score, precision_score

In [ ]:
def make_mbt_tfidf(Xtr,Xv,Xt):
    """
    The function `make_tfidf` takes in three sets of data (train, validation, and test) and applies the
    TF-IDF vectorization technique to convert the text data into numerical features, using n-grams up to
    trigrams and keeping single characters. It then returns the transformed data as pandas DataFrames.
    
    :param Xtr: Xtr is the training data, which is a dataframe containing the text data that you want to
    transform into TF-IDF features. The "lemmatized" column in the dataframe contains the preprocessed
    text data
    :param Xv: Xv is the validation dataset, which is used to evaluate the performance of the model
    during training
    :param Xt: Xt is the input data for the test set. It is a dataframe containing the text data that
    needs to be transformed into TF-IDF representation
    :return: three dataframes: Xtr_tfidf, Xv_tfidf, and Xt_tfidf.
    """
    #make my bag of words up to trigrams tfidf and keep single characters
    tfidf = TfidfVectorizer(token_pattern=r'(?u)\b\w+\b',lowercase=False, max_features=10000, ngram_range=(1,3))
    # fit and transform train
    Xtr_bow_tfidf = tfidf.fit_transform(Xtr.lemon.astype(str))
    # transform val and test
    Xv_bow_tfidf = tfidf.transform(Xv.lemon.astype(str))
    Xt_bow_tfidf = tfidf.transform(Xt.lemon.astype(str))
    # make dfs
    Xtr_tfidf = pd.DataFrame(Xtr_bow_tfidf.todense(),columns=tfidf.get_feature_names_out(),index=Xtr.index)
    Xv_tfidf = pd.DataFrame(Xv_bow_tfidf.todense(),columns=tfidf.get_feature_names_out(),index=Xv.index)
    Xt_tfidf = pd.DataFrame(Xt_bow_tfidf.todense(),columns=tfidf.get_feature_names_out(),index=Xt.index)
    return Xtr_tfidf,Xv_tfidf,Xt_tfidf

def process_vector_merge(X_train,X_val,X_test):
    X_train_tf, X_val_tf, X_test_tf = make_mbt_tfidf(X_train[['lemon']], X_val[['lemon']], X_test[['lemon']])
    
    encoded_train = X_train.iloc[:, 1:]
    encoded_val = X_val.iloc[:, 1:]
    encoded_test = X_test.iloc[:, 1:]
    
    X_train_tfe = encoded_train.merge(X_train_tf, left_index=True, right_index=True)
    X_val_tfe = encoded_val.merge(X_val_tf, left_index=True, right_index=True)
    X_test_tfe = encoded_test.merge(X_test_tf, left_index=True, right_index=True)
    
    # Visualization of train data
    return X_train_tfe, X_val_tfe, X_test_tfe

In [ ]:
# vectorize
X_train_tfe, X_val_tfe, X_test_tfe = process_vector_merge(X_train,X_val,X_test)
X_train_tfe

In [ ]:
# make
dt = DecisionTreeClassifier(random_state=123, max_depth=9, max_leaf_nodes=11)

# fit
dt.fit(X_train_tfe, y_train)

# use
dt_train = dt.score(X_train_tfe, y_train)
dt_val = dt.score(X_val_tfe, y_val)
print(f"Train Accuracy: {dt_train:.3}%")
print(f"Validate Accuracy: {dt_val:.3}%")

#predict
dt_pred_train = dt.predict(X_train_tfe)
dt_pred_val = dt.predict(X_val_tfe)

# recall score
r_score_train = recall_score(y_train, dt_pred_train, pos_label='no_relief')
r_score_val = recall_score(y_val, dt_pred_val, pos_label='no_relief')
print(f"Train recall score: {r_score_train:.3}%")
print(f"Validate recall score: {r_score_val:.3}%")

In [ ]:
# # classification report for train
pd.DataFrame(classification_report(y_train, dt_pred_train, output_dict=True))

In [ ]:
# # classification report for validate
pd.DataFrame(classification_report(y_val, dt_pred_val, output_dict=True))

In [ ]:
# make
dt = DecisionTreeClassifier(random_state=123, max_depth=25, max_leaf_nodes=30)

# fit
dt.fit(X_train_tfe, y_train)

# use
dt_train = dt.score(X_train_tfe, y_train)
dt_val = dt.score(X_val_tfe, y_val)
print(f"Train Accuracy: {dt_train:.3}%")
print(f"Validate Accuracy: {dt_val:.3}%")

#predict
dt_pred_train = dt.predict(X_train_tfe)
dt_pred_val = dt.predict(X_val_tfe)

# recall score
r_score_train = recall_score(y_train, dt_pred_train, pos_label='no_relief')
r_score_val = recall_score(y_val, dt_pred_val, pos_label='no_relief')
print(f"Train recall score: {r_score_train:.3}%")
print(f"Validate recall score: {r_score_val:.3}%")

In [ ]:
# # classification report for train
pd.DataFrame(classification_report(y_train, dt_pred_train, output_dict=True))

In [ ]:
# # classification report for validate
pd.DataFrame(classification_report(y_val, dt_pred_val, output_dict=True))

In [ ]:
# train a logistic regression model on the training set
from sklearn.linear_model import LogisticRegression

# confusion matrix
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

# plot 
import matplotlib.pyplot as plt

In [ ]:

# make
lr = LogisticRegression(random_state=123)

# fit model
lr.fit(X_train_tfe, y_train)

# use
lr_train = lr.score(X_train_tfe, y_train)
lr_val = lr.score(X_val_tfe, y_val)
print(f"Train Accuracy: {lr_train:.3}%")
print(f"Validate Accuracy: {lr_val:.3}%")

# predictions
y_pred_train = lr.predict(X_train_tfe)
y_pred_val = lr.predict(X_val_tfe)

# recall score
r_score_train = recall_score(y_train, y_pred_train, pos_label='no_relief')
r_score_val = recall_score(y_val, y_pred_val, pos_label='no_relief')
print(f"Train recall score: {r_score_train:.3}%")
print(f"Validate recall score: {r_score_val:.3}%")

In [ ]:
# classification report for train
round(pd.DataFrame(classification_report(y_train, y_pred_train, output_dict=True)),3)

In [ ]:
# classification report for validate
round(pd.DataFrame(classification_report(y_val, y_pred_val, output_dict=True)),3)

In [ ]:
# confusion matrix
ConfusionMatrixDisplay(confusion_matrix(y_train, y_pred_train, labels=lr.classes_), display_labels=lr.classes_).plot()
plt.show()

In [ ]:
# establish TN, TP, FP, FN
TN = 4433
TP = 115203
FP = 2672
FN = 26317

In [ ]:
# specificity
specificity = TN / (TN + FP)

print(specificity)

minimizing damage by looking at additional metrics on top of accuracy baseline which is an average. Slightly improved on baseline but where we really saw the benefit was looking at the recall score which states predicts relief - no relief given.

98% good at catching false negatives. The times that isn't relieved results in 98% catch of this error which means, happier consumers, happier businesses.

In [ ]:
# make
lr = LogisticRegression(random_state=123)
# fit model
lr.fit(X_train_tfe, y_train)
# use
lr_train = lr.score(X_train_tfe, y_train)
lr_val = lr.score(X_val_tfe, y_val)
print(f"Train Accuracy: {lr_train:.3}%")
print(f"Validate Accuracy: {lr_val:.3}%")
# predictions
y_pred_train = lr.predict(X_train_tfe)
y_pred_val = lr.predict(X_val_tfe)
# recall score
r_score_train = recall_score(y_train, y_pred_train, pos_label='no_relief')
r_score_val = recall_score(y_val, y_pred_val, pos_label='no_relief')
print(f"Train recall score: {r_score_train:.3}%")
print(f"Validate recall score: {r_score_val:.3}%")

In [ ]:
# # test

# # use
# lr_train = lr.score(X_train_tfe, y_train)
# lr_val = lr.score(X_val_tfe, y_val)
# print(f"Train Accuracy: {lr_train:.3}%")
# print(f"Validate Accuracy: {lr_val:.3}%")

# # predictions
# y_pred_train = lr.predict(X_train_tfe)
# y_pred_val = lr.predict(X_val_tfe)

# # recall score
# r_score_train = recall_score(y_train, y_pred_train, pos_label='no_relief')
# r_score_val = recall_score(y_val, y_pred_val, pos_label='no_relief')
# print(f"Train recall score: {r_score_train:.3}%")
# print(f"Validate recall score: {r_score_val:.3}%")

In [16]:
non_string_elements = X_train['lemon'].apply(lambda x: isinstance(x, str) == False)
print(X_train[non_string_elements])

       lemon  Older American  Older American, Servicemember  Servicemember  \
7137    None               0                              0              0   
20876   None               0                              0              0   
42286   None               0                              0              0   
45663   None               0                              0              0   
100459  None               0                              0              0   

        credit_card  credit_report  debt_collection  loans  money_service  \
7137              0              1                0      0              0   
20876             0              1                0      0              0   
42286             0              0                1      0              0   
45663             0              1                0      0              0   
100459            0              1                0      0              0   

        mortgage  
7137           0  
20876          0  
42286      

In [18]:
X_train.value_counts()


lemon                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  Older American  Older American, Servicemember  Servicemember  credit_card  credit_report  debt_collection  loans  money_service  mortgage
accordance fair credit reporting act account violated right u c section state right privacy u c section section also state consumer reporting agency furnish account without written instruction                                                                                                       

In [23]:
X_train.isnull().value_counts()

lemon  Older American  Older American, Servicemember  Servicemember  credit_card  credit_report  debt_collection  loans  money_service  mortgage
False  False           False                          False          False        False          False            False  False          False       148620
True   False           False                          False          False        False          False            False  False          False            5
dtype: int64

In [17]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

train_encodings = tokenizer(list(X_train['lemon'].values), truncation=True, padding=True)
val_encodings = tokenizer(list(X_val['lemon'].values), truncation=True, padding=True)
test_encodings = tokenizer(list(X_test['lemon'].values), truncation=True, padding=True)

ValueError: Input is not valid. Should be a string, a list/tuple of strings or a list/tuple of integers.

In [ ]:
from transformers import BertForSequenceClassification, Trainer, TrainingArguments

model = BertForSequenceClassification.from_pretrained('bert-base-uncased')

# Define the training arguments
training_args = TrainingArguments(
    output_dir='./results',    # output directory
    num_train_epochs=3,        # total number of training epochs
    per_device_train_batch_size=16,  # batch size per device during training
    per_device_eval_batch_size=64,   # batch size for evaluation
    warmup_steps=500,          # number of warmup steps for learning rate scheduler
    weight_decay=0.01,         # strength of weight decay
)

trainer = Trainer(
    model=model,               # the instantiated 🤗 Transformers model to be trained
    args=training_args,        # training arguments, defined above
    train_dataset=X_train,  # training dataset
    eval_dataset=X_val,   # evaluation dataset
)

trainer()

In [ ]:
from transformers import BertForSequenceClassification, BertTokenizerFast, Trainer, TrainingArguments

# Load pre-trained model and tokenizer
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=5)
tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')

# Tokenize and prepare the datasets for BERT
train_encodings = tokenizer(train_texts, truncation=True, padding=True)
val_encodings = tokenizer(val_texts, truncation=True, padding=True)
test_encodings = tokenizer(test_texts, truncation=True, padding=True)

# Convert to PyTorch Datasets
train_dataset = torch.utils.data.Dataset(train_encodings, train_labels)
val_dataset = torch.utils.data.Dataset(val_encodings, val_labels)
test_dataset = torch.utils.data.Dataset(test_encodings, test_labels)

# Define the training arguments
training_args = TrainingArguments(
    output_dir='./results',  # output directory
    num_train_epochs=3,  # total number of training epochs
    per_device_train_batch_size=16,  # batch size per device during training
    per_device_eval_batch_size=64,  # batch size for evaluation
    warmup_steps=500,  # number of warmup steps for learning rate scheduler
    weight_decay=0.01,  # strength of weight decay
    logging_dir='./logs',  # directory for storing logs
    logging_steps=10,
)

# Create a trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset
)

# Train the model
trainer.train()


In [ ]:
import torch
x = torch.rand(5, 3)
print(x)

In [ ]:
pip install transformers[torch]

In [ ]:
X_train = X_train.reset_index(drop=True)
X_val = X_val.reset_index(drop=True)

In [ ]:
from torch.utils.data import Dataset

class CustomDataset(Dataset):

    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

# Then, you can use this class to create your datasets:
train_dataset = CustomDataset(train_encodings, train_labels)
val_dataset = CustomDataset(val_encodings, val_labels)

# Now, you can pass these to the Trainer:
trainer = Trainer(
    model=model,               # the instantiated 🤗 Transformers model to be trained
    args=training_args,        # training arguments, defined above
    train_dataset=train_dataset,  # training dataset
    eval_dataset=val_dataset,   # evaluation dataset
)


In [ ]:
train

In [ ]:
X_train

In [ ]:
X_train['lemon'] = X_train['lemon'].fillna('')
X_val['lemon'] = X_val['lemon'].fillna('')
X_test['lemon'] = X_test['lemon'].fillna('')

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

# Define the features (X) and the target variable (y)
X_train = train['lemon']
y_train = train['response']
X_test = test['lemon']
y_test = test['response']


X_train['lemon'] = X_train['lemon'].fillna('')
X_val['lemon'] = X_val['lemon'].fillna('')
X_test['lemon'] = X_test['lemon'].fillna('')

X_train = X_train.reset_index(drop=True)
X_val = X_val.reset_index(drop=True)

# Preprocess the text data (e.g., vectorize the text using CountVectorizer or TF-IDFVectorizer)

# Create an instance of the Gradient Boosting Classifier
gbc = GradientBoostingClassifier()

# Fit the model on the training data
gbc.fit(X_train, y_train)

# Make predictions on the test data
y_pred = gbc.predict(X_test)

# Evaluate the model's performance
accuracy = gbc.score(X_test, y_test)
print("Accuracy:", accuracy)

# Perform hyperparameter tuning (optional)
# You can use techniques like grid search or randomized search to find the optimal hyperparameters

# Example of hyperparameter tuning using GridSearchCV
from sklearn.model_selection import GridSearchCV

param_grid = {
    'n_estimators': [100, 200, 300],
    'learning_rate': [0.1, 0.05, 0.01],
    'max_depth': [3, 4, 5]
}

grid_search = GridSearchCV(gbc, param_grid, cv=5)
grid_search.fit(X_train, y_train)

# Print the best parameters found during hyperparameter tuning
print("Best Parameters:", grid_search.best_params_)

# Make predictions using the best model
best_model = grid_search.best_estimator_
y_pred_best = best_model.predict(X_test)

# Evaluate the performance of the best model
accuracy_best = best_model.score(X_test, y_test)
print("Best Model Accuracy:", accuracy_best)


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

# Create an instance of CountVectorizer
vectorizer = CountVectorizer()

# Fit the vectorizer on the training data
X_train_vectorized = vectorizer.fit_transform(X_train)

# Transform the test data using the fitted vectorizer
X_test_vectorized = vectorizer.transform(X_test)

# Create an instance of the Gradient Boosting Classifier
gbc = GradientBoostingClassifier()

# Fit the model on the vectorized training data
gbc.fit(X_train_vectorized, y_train)

# Make predictions on the vectorized test data
y_pred = gbc.predict(X_test_vectorized)

# Evaluate the model's performance
accuracy = gbc.score(X_test_vectorized, y_test)
print("Accuracy:", accuracy)


In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

# Define the features (X) and the target variable (y)
X_train = train['lemon']
y_train = train['response']
X_test = test['lemon']
y_test = test['response']

X_train = X_train.fillna('')
X_test = X_test.fillna('')

X_train = X_train.reset_index(drop=True)
X_test = X_test.reset_index(drop=True)

# Preprocess the text data (e.g., vectorize the text using CountVectorizer or TF-IDFVectorizer)

# Create an instance of the Gradient Boosting Classifier
gbc = GradientBoostingClassifier()

# Fit the model on the training data
gbc.fit(X_train, y_train)

# Make predictions on the test data
y_pred = gbc.predict(X_test)

# Evaluate the model's performance
accuracy = gbc.score(X_test, y_test)
print("Accuracy:", accuracy)

# Perform hyperparameter tuning (optional)
# You can use techniques like grid search or randomized search to find the optimal hyperparameters

# Example of hyperparameter tuning using GridSearchCV
from sklearn.model_selection import GridSearchCV

param_grid = {
    'n_estimators': [100, 200, 300],
    'learning_rate': [0.1, 0.05, 0.01],
    'max_depth': [3, 4, 5]
}

grid_search = GridSearchCV(gbc, param_grid, cv=5)
grid_search.fit(X_train, y_train)

# Print the best parameters found during hyperparameter tuning
print("Best Parameters:", grid_search.best_params_)

# Make predictions using the best model
best_model = grid_search.best_estimator_
y_pred_best = best_model.predict(X_test)

# Evaluate the performance of the best model
accuracy_best = best_model.score(X_test, y_test)
print("Best Model Accuracy:", accuracy_best)
